# Исследование надёжности заёмщиков

## Вступление

**Заказчик** — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Данные

Датасет представлен ввиде файла с расширением csv (data.csv).
Путь к файлу: /datasets/data.csv

Датасет включает в себя следующие колонки:

    children — количество детей в семье
    days_employed — общий трудовой стаж в днях
    dob_years — возраст клиента в годах
    education — уровень образования клиента
    education_id — идентификатор уровня образования
    family_status — семейное положение
    family_status_id — идентификатор семейного положения
    gender — пол клиента
    income_type — тип занятости
    debt — имел ли задолженность по возврату кредитов
    total_income — ежемесячный доход
    purpose — цель получения кредита

## Задачи исследования

### 1. Изучить информацию
    - открыть файл
    - просмотреть начало и конец датасета
    - просмотреть каждую колонку в отдельности
    - сделать первые выводы

### 2. Предобработка данных
    - обработка пропусков
    - изменить типы данных
    - обработать дубликаты
    - выделить леммы
    - провести категоризацию данных

### 3. Ответы на поставленные задачи
    - Есть ли зависимость между наличием детей и возвратом кредита в срок?
    - Есть ли зависимость между семейным положением и возвратом кредита в срок?
    - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    - Как разные цели кредита влияют на его возврат в срок?

### 4. Результаты
    - Вывод
    - Рекомендации

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
# импорт библиотеки pandas
# импорт библиотеки pymystem3 для лемматизации
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

# чтение файла
data = pd.read_csv('/datasets/data.csv')
# общая информация о данных

data.info()

# первые 10 строк таблицы
display(data.head(10))

# последние 10 строк таблицы
display(data.tail(10))

# названия сталбцов оборачиваем в лист
columns = list(data)

# проходим в цикле по всем столбцам и считаем количество каждого значения в столбце
for column in columns:
    print('\n', '-' * 122)
    print(column)
    data.sort_values(column)
    print(data[column].value_counts().sort_values(ascending=False))

# исправляем название столбцов для простоты понимания. (Пример: total_income это общий доход,а описан как месячный)
data.rename(columns={'days_employed': 'days_employed_in_year', 'dob_years': 'years_old', 'income_type': 'type_employment',\
                       'total_income': 'monthly_income'}, inplace=True)

print('\n', '-' * 122)

# проверим наименования столбцов
display(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля



 --------------------------------------------------------------------------------------------------------------------------
children
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

 --------------------------------------------------------------------------------------------------------------------------
days_employed
-582.538413       1
-545.115232       1
-628.799246       1
-5154.422247      1
-805.590053       1
                 ..
-1283.436610      1
-4645.841330      1
-3924.380059      1
 388367.367456    1
-986.927316       1
Name: days_employed, Length: 19351, dtype: int64

 --------------------------------------------------------------------------------------------------------------------------
dob_years
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508


,children,days_employed_in_year,years_old,education,education_id,family_status,family_status_id,gender,type_employment,debt,monthly_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Спасибо за подсказку. С display гараздо легче воспринимать информацию (когда выводил через print, мучался).

### Вывод

После просмотра информации было выявленно следующее:
1. В колонках days_employed и total_income обнаружены NaN и None. Необходимо заполнить 0 так, как колонки отображают стаж и доход, возможно данные люди безработные. Для подтверждения данной теории необходимо вывести данные колонки совместно с колонкой income_type.
2. Строки разного регистра в колонке education, необходимо привести к одному регистру.
3. Колонка days_employed отрицательные значения. Данные представлены в днях, не понятен стаж, необходимо перевести в года.
4. Колонка total_income имеет тип данных float, сумму ежемесячного дохода можно округлить до целого числа переведя в тип данных int так, как копейки не значительны и не окажут влияния на данные.
5. В колонке children отрицательное значение.
6. В колонке gender, неизвестное значение XNA, данное значение можно удалить так, как количество таких значений не значительно и не повлияет ни на расчеты, ни на выводы
7. Колонка purpose требует применения лемматизации.
8. Изменить столбцы на более читаемые
9. В столбце возраст, есть клиенты с возрастом 0.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
# проверяем количество пропусков
print(data.isnull().sum())
print('\n', '-' * 122)

# выводим количество вхождений по каждой должности в колонке income_type
# для каждого клиента с отсутствующим доходом и стажем
print(data.loc[(data['days_employed_in_year'].isnull()) & (data['monthly_income'].isnull())]['type_employment'].value_counts())

print('\n', '-' * 122)

# групперуем по колонке type_employment и сохраняем в переменной
type_employment_group = data.groupby('type_employment')

# колонку monthly_income заполняем медианным значением
data['monthly_income'] = type_employment_group['monthly_income'].apply(lambda monthly_income_median:\
                                                      monthly_income_median.fillna(monthly_income_median.median()))

# колонку days_employed_in_year заполняем средним значением
data['days_employed_in_year'] = type_employment_group['days_employed_in_year'].apply(lambda monthly_income_median:\
                                                      monthly_income_median.fillna(monthly_income_median.mean()))

# проверяем заполненость пропусков
print(data.isnull().sum())

print('\n', '-' * 122)

# проверяем количество несовершеннолетних
print(data.loc[data['years_old']<18].count())

# переводим стаж в годы
data['days_employed_in_year'] = data['days_employed_in_year'] / 365

children                    0
days_employed_in_year    2174
years_old                   0
education                   0
education_id                0
family_status               0
family_status_id            0
gender                      0
type_employment             0
debt                        0
monthly_income           2174
purpose                     0
dtype: int64

 --------------------------------------------------------------------------------------------------------------------------
сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: type_employment, dtype: int64

 --------------------------------------------------------------------------------------------------------------------------
children                 0
days_employed_in_year    0
years_old                0
education                0
education_id             0
family_status            0
family_status_id         0
gender                   0
type_

### Вывод

    Количество пропусков в колонках days_employed_in_year и monthly_income одинакого. Первое предположение: безработные, но в колонке с должностями нет пропусков. Проверя столбец с должностями отфильтровав по колонкам days_employed_in_year и monthly_income в которых сделаны пропуски, можно предположить, что выгрузка данных произведена с ошибками, либо сотрудник по каким то причинам не заполнил данные по доходу и стажу. Возможно система пропустила, либо сотрудник забыл(а) внести данные.
    Необходимо проверить параметры выгрузки
    Необходимо проверить теорию о том, что система выдает кредит клиентам без заполненых данных о доходах и стаже.
    Было выявлено, что кредиты выдавались не совершеннолетним клиентам.
    Необходимо протестировать и настроить проверку на совершеннолетие клиента и установить запрет выдачи кредита

#### Данные

    Пропуски были заполнены медианным значением в колонке monthly_income в соответствии с типом занятости
    Пропуски были заполнены средним значением в колонке days_employed_in_year в соответствии с типом занятости

### Замена типа данных

In [3]:
# изменяем тип данных в столбце monthly_income с float на int
data['monthly_income'] = data['monthly_income'].astype('int')
data['days_employed_in_year'] = data['days_employed_in_year'].astype('int')

# проверяем изменение типа данных и сортируем список по убыванию
print(data[['monthly_income', 'days_employed_in_year']].head(10))


   monthly_income  days_employed_in_year
0          253875                    -23
1          112080                    -11
2          145885                    -15
3          267628                    -11
4          158616                    932
5          255763                     -2
6          240525                     -7
7          135823                      0
8           95856                    -18
9          144425                     -5


### Обработка дубликатов

In [4]:
# Проверяем количество дубликатов
print('Количество дублекатов:', data.duplicated().sum())

# удаляем дубликаты и проверяем
data = data.drop_duplicates().reset_index(drop=True)
print('Количество дублекатов:', data.duplicated().sum())

print('\n', '-' * 122)

# переводим в нижний регистр колонку education
data['education'] = data['education'].str.lower()

# конвертируем отрицательные числа
data['children'] = data['children'].abs()
data['days_employed_in_year'] = data['days_employed_in_year'].abs()

# проверям
print(data['children'].value_counts())
print('\n', '-' * 122)
print(data['days_employed_in_year'].value_counts())
print('\n', '-' * 122)
display(data.head())

Количество дублекатов: 54
Количество дублекатов: 0

 --------------------------------------------------------------------------------------------------------------------------
0     14107
1      4856
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64

 --------------------------------------------------------------------------------------------------------------------------
6     2042
1     2024
2     1878
0     1827
3     1560
      ... 
42       2
44       2
50       1
48       1
45       1
Name: days_employed_in_year, Length: 249, dtype: int64

 --------------------------------------------------------------------------------------------------------------------------


,children,days_employed_in_year,years_old,education,education_id,family_status,family_status_id,gender,type_employment,debt,monthly_income,purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,932,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

    Возможно данный датасет был создан из объединения несколько датасетов и образовались дубли
    Неисключаю вероятность ошибки при выгрузке
    Человеческий фактор (дважды и более внесеные данные)

### Лемматизация

In [5]:
m = Mystem()

print(data['purpose'].value_counts())
print('\n', '-' * 122)

# Лемматизируем значения столбца purpose
unique_purpose = data['purpose'].unique().tolist()
print(unique_purpose)
string = ' '.join(unique_purpose)
lemmas = m.lemmatize(string)
print(lemmas)
print('\n', '-' * 122)
print(Counter(lemmas))

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

### Вывод
    Топ 5 целей кредита:
    Недвижимость, жилье
    Автомобиль
    Образование

### Категоризация данных

In [6]:
# создаем два словаря data_dict_education и data_dict_family_status

data_dict_education = data[['education', 'education_id']]
data_dict_education = data_dict_education.drop_duplicates().reset_index(drop=True)
print(data_dict_education.sort_values('education_id'))

print('\n', '-' * 122)

data_dict_family_status = data[['family_status', 'family_status_id']]
data_dict_family_status = data_dict_family_status.drop_duplicates().reset_index(drop=True)
print(data_dict_family_status.sort_values('family_status_id'))

             education  education_id
0               высшее             0
1              среднее             1
2  неоконченное высшее             2
3            начальное             3
4       ученая степень             4

 --------------------------------------------------------------------------------------------------------------------------
           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  Не женат / не замужем                 4


### Вывод

Выделены словари data_dict_education и data_dict_family_status. Облегчит процесс рашения поставленных задач

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
# создаем сводную таблицу
children_pivot = data.pivot_table(index = ['children'], columns='debt',\
                                                values='years_old', aggfunc='count')

# считаем процент клиентов, которые не отдали в срок кредит
children_pivot['%_debt'] = children_pivot[1] \
/ (children_pivot[1] + children_pivot[0]) * 100

# Выводим данные сводной таблицы для анализа
print(children_pivot)
print('\n', '-' * 122)

# проверяем людей имеет 20 детей с учетом возраста (расчет на то, что клиент женщина родившая первого ребенка в 19 лет)
display(data[(data['children'] == 20) & (data['years_old'] <= 39)\
                    & (data['gender'] <= 'F')])

debt            0       1     %_debt
children                            
0         13044.0  1063.0   7.535266
1          4411.0   445.0   9.163921
2          1858.0   194.0   9.454191
3           303.0    27.0   8.181818
4            37.0     4.0   9.756098
5             9.0     NaN        NaN
20           68.0     8.0  10.526316

 --------------------------------------------------------------------------------------------------------------------------


,children,days_employed_in_year,years_old,education,education_id,family_status,family_status_id,gender,type_employment,debt,monthly_income,purpose
2940,20,5,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739,на покупку автомобиля
3301,20,9,35,среднее,1,Не женат / не замужем,4,F,госслужащий,0,150447,профильное образование
3670,20,2,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,101255,на покупку подержанного автомобиля
5312,20,5,24,среднее,1,женат / замужем,0,F,сотрудник,0,100415,покупка коммерческой недвижимости
10725,20,11,30,высшее,0,женат / замужем,0,F,компаньон,0,154706,высшее образование
11476,20,1,34,среднее,1,женат / замужем,0,F,сотрудник,0,389841,покупка жилой недвижимости
12374,20,2,31,высшее,0,женат / замужем,0,F,компаньон,0,181325,жилье
13273,20,12,38,среднее,1,женат / замужем,0,F,сотрудник,0,69212,дополнительное образование
14451,20,7,27,среднее,1,женат / замужем,0,F,сотрудник,0,244092,высшее образование
15647,20,14,33,среднее,1,гражданский брак,1,F,сотрудник,0,155567,свадьба


### Вывод

1. Сводная таблица показывает, что семьи с 20 детьми, отдают кредиты хуже всего, но эти данные я бы не брал в рассчет. Понимаю, что могут быть приемные дети, от нескольких браков и т.д., но не смотря на это количество таких клиентов слишком мало, что бы сделать какие либо выводы по данной группе клиентов.
2. Клиент с 5 детьми не имеют просрочек и являются лучшеми клиентами для банка, но таких клиентов слишком мало, что бы сделать какие либо выводы по данной группе клиентов.
3. Клиенты с 4, 3 детьми так же ситуация, что с 5 и 20, мало данных
4. Взяв наиболее многочисленые группы, семьи с 1, 2 детьми и без детей, наглядно видно связь между количеством детей и возвратом кредитов в срок и группа клиентов без детей возвращают кредиты охотнее.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
# создаем сводную таблицу
family_status_pivot = data.pivot_table(index = ['family_status'], columns='debt',\
                                                values='years_old', aggfunc='count')

# считаем процент
family_status_pivot['%_debt'] = family_status_pivot[1] \
/ (family_status_pivot[1] + family_status_pivot[0]) * 100

# Выводим данные сводной таблицы для анализа
print(family_status_pivot)

debt                       0    1    %_debt
family_status                              
Не женат / не замужем   2536  274  9.750890
в разводе               1110   85  7.112971
вдовец / вдова           896   63  6.569343
гражданский брак        3775  388  9.320202
женат / замужем        11413  931  7.542126


### Вывод
Клиенты не женатые / не замужем и находящиеся в гражданском браке (сожительство) не охотно возвращают кредиты, Возможно это связано с тем, что данная группа людей чаще всего испытывает финансовые трудности, в то время как клиенты находящиеся в разводе или вдовы / вдовцы отдают кредиты в срок, в данном случае люди раведеные получают часть совместно нажитого имущества, которое распродается для погашения кредита, вдовы / вдовцы, в связи с утратой, избавляются от вещей и направляют вырученые средства на погашение кредита, так же возможна финансовая поддержка родственников.
Наимобее стабильная группа, с средними показателями - это клиенты женатые / замужем, тут на мой взгляд все очевидно, есть кормилец, либо оба работающие в семье, что облегчает финансовую нагрузку.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
# делим доход на квантили

monthly_income_quintile = pd.qcut(data['monthly_income'], 4, labels=['низкий', 'средний', 'высокий', 'очень высокий'])

data['quintile'] = monthly_income_quintile

def quintile_id(row):
    
    quintile_row = row['quintile']

    if quintile_row == 'очень высокий':
        return 3
    elif quintile_row == 'высокий':
        return 2
    elif quintile_row == 'средний':
        return 1
    return 0


row_values = ['quintile']
row_columns = ['quintile']
row = pd.Series(data=row_values, index=row_columns)
data['quintile_id'] = data.apply(quintile_id, axis=1)

display(data.head(10))

# в функции проверяем ключевые слова
def purpose_check(purpose):
    if 'образов' in purpose:
        return 'образование'
    elif 'авто' in purpose:
        return 'автомобиль'
    elif 'недвиж' or 'жиль' in purpose:
        return 'недвижимость'
    else:
        return 'нет цели'

data['purpose_check'] = data['purpose'].apply(purpose_check)

# создаем сводную таблицу
income_level_pivot = data.pivot_table(index=['quintile'], columns='debt', values='years_old', aggfunc='count')

# считаем процент
income_level_pivot['%_debt'] = income_level_pivot[1] / (income_level_pivot[1] + income_level_pivot[0]) * 100

# Выводим данные сводной таблицы для анализа
print(income_level_pivot)

print('\n', '-' * 122)

print(data.groupby('quintile')['purpose_check'].value_counts())

,children,days_employed_in_year,years_old,education,education_id,family_status,family_status_id,gender,type_employment,debt,monthly_income,purpose,quintile,quintile_id
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,очень высокий,3
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,средний,1
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,высокий,2
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,очень высокий,3
4,0,932,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,высокий,2
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,очень высокий,3
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,очень высокий,3
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,средний,1
8,2,18,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,низкий,0
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,высокий,2


debt              0    1    %_debt
quintile                          
низкий         4941  427  7.954545
средний        5007  483  8.797814
высокий        4797  448  8.541468
очень высокий  4985  383  7.134873

 --------------------------------------------------------------------------------------------------------------------------
quintile       purpose_check
низкий         недвижимость     3279
               автомобиль       1046
               образование      1043
средний        недвижимость     3311
               автомобиль       1155
               образование      1024
высокий        недвижимость     3226
               автомобиль       1016
               образование      1003
очень высокий  недвижимость     3333
               автомобиль       1091
               образование       944
Name: purpose_check, dtype: int64


### Вывод

    Связь между доходом и возвратом кредита в срок есть, исключение клиенты с низким доходом.
    
    Лучший показатель у клиентов с очень высокими доходами - это очевидно.
    Клиенты с самым низким уровнем дохода так же показали хороший результат. Связываю это с тем, что кредит берется по потребностям и по возможностям, клиенты с низким доходом не будут рисковать и брать кредит больше, чем они реально могут погасить.
    Обратная сторона с клиентами с средним и высоким доходами. Они обычно берут кредит не спопстовимый с финансовыми возможностями, отсюда и худший результат.

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
# создаем сводную таблицу
purpose_pivot = data.pivot_table(index = ['purpose_check'], columns='debt',\
                                                values='years_old', aggfunc='count')

# считаем процент
purpose_pivot['%_debt'] = purpose_pivot[1] / (purpose_pivot[1] + purpose_pivot[0]) * 100

# Выводим данные сводной таблицы для анализа
print(purpose_pivot)

debt               0    1    %_debt
purpose_check                      
автомобиль      3905  403  9.354689
недвижимость   12181  968  7.361777
образование     3644  370  9.217738


### Вывод
1. Наихудший результат у клиентов берущих кредиты на образование и автомобили, возможно связано с тем, что бросают учится и разбивают авто и платить за 'воздух' не хотят.
2. Недвижимость одни из важнейших приоритетов у человека, люди охотнее выплачивают данные кредиты

### Шаг 4. Общий вывод

    Проанализировав данные можно сказать, что между наличием детей, целью, уровнем дохода и возвратом кредита в срок, есть связь, хотя и разница в процентном соотношении мала. Так же утверждать, что анализ верен можно только с данными у которых большое количество обращений (пример: недвижимость, кредит брали 8249 раз), обратный пример (количество выданых кредитов крайне мало) показывает, что клиент с большим количество детей отдают кредиты в срок, этого утверждать мы не можем так, как было всего 8 обращений данных клиентов в банк (пример: 5 детей, кредит брали 8 раз).

#### Рекомендации
1. Нет уникальных значений, необходима колонка с уникальными ID клиентов для поиска дублей
2. Необходима валидация данных при заполнени формы, анкеты клиента, что бы исключить возможность установки отрицательных значений в колонке children, 0 в колонке years_old, все поля должны быть в одном регистре, сделать жестче выбор цели кредита (пример: выпадающий список). Все перечисленное упростит работу с данными.
3. Доход округлять до целых чисел (копейки не имеют значения)
4. Стаж указывать в годах (пример: 1 - год, 1,5 - года)

#### Проделанная работа
1. Изучен датасет
2. Выявлены проблемы данных
3. Решение проблем было методом:

    - обработки пропусков
    - изменения типов данных
    - обработки дубликатов
    - выделения леммы
    - проведения категоризации данных
4. Даны ответы на поставленные задачи